In [1]:
import tensorflow as tf
import os
import math
import time
import pickle
from PIL import Image
import numpy as np
from vgg16 import Vgg16
import cifar10_input
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore")
CIFAR_DIR = "./cifar-10-batches-py/"
#print (os.listdir(CIFAR_DIR))
bat_data, bat_labels = cifar10_input.distorted_inputs(data_dir=CIFAR_DIR,batch_size=20)
print(bat_data)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLe

In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],strides=[1, 2, 2, 1], padding='SAME')

    

In [4]:
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 -1
        self._labels = np.hstack(all_labels)
        '''print (self._data.shape)
        print (self._labels.shape)
        '''
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        #[0,1,2,3,4,5]-> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples  ")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

                            
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d.bin' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch.bin')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)
a,b = train_data.next_batch(20)


In [5]:
print(a.shape)

(20, 3072)


In [7]:

'''
# (3072,10)
w = tf.get_variable('w',[x.get_shape()[-1], 10],
                    initializer = tf.random_normal_initializer(0, 1))
#(10, )
b = tf.get_variable('b',[10],
                    initializer = tf.constant_initializer(0.0))

# [None,3072]*[3072,10] = [None,10]
y_ = tf.matmul(x,w) + b
'''
# (3072,10)
#W_conv1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, w1=0.0)

#(10, )
#b_conv1 = tf.Variable(tf.constant(0.0, shape=[64]))
batch_size = 20


x = tf.placeholder(tf.float32,[20,3072])
#[None]
y = tf.placeholder(tf.int64,[batch_size])

W_conv1 = weight_variable([5, 5, 3, 64])
b_conv1 = bias_variable([64])
x_image = tf.reshape(x, [20,32,32,3])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
norm1 = tf.nn.lrn(h_pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)

W_conv2 = weight_variable([5, 5, 64, 64])

b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
norm2 = tf.nn.lrn(h_conv2, 4, bias=1.0, alpha=0.001/0.9, beta=0.75)

h_pool2_flat = tf.reshape(h_pool2, [batch_size, -1])
dim = h_pool2_flat.get_shape()[1].value
W_fc1 = weight_variable([dim, 384])
b_fc1 = bias_variable([384])

h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

W_fc2 = weight_variable([384, 192])
b_fc2 = bias_variable([192])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

keep_prob = tf.placeholder(tf.float32)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

W_fc3 = weight_variable([192, 10])
b_fc3 = bias_variable([10])
y_ = tf.matmul(h_fc2_drop, W_fc3) + b_fc3
#y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
print(x)
print(y_)
'''

#
p_y = vgg.forward(images)
#5 -> [0,0,0,0,0,1,0,0,0,0]
y_one_hot = tf.one_hot(y,10,dtype=tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
'''


loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
#y_ -> sofmax
#y -> one_hot
#loss = ylogy_
#bool
predict = tf.argmax(y_,1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))
top_k_op = tf.nn.in_top_k(y_, y, 1)
#image_prediction = tf.nn.top_k(y_,1)
#image_real = tf.nn.top_k(y,1)
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)


Tensor("Placeholder_3:0", shape=(20, 3072), dtype=float32)
Tensor("add_9:0", shape=(20, 10), dtype=float32)


In [10]:
init = tf.global_variables_initializer()
train_steps = 3000
test_steps = 100
#test_batch_data, test_batch_labels = cifar10_input.inputs(eval_data=True, data_dir=CIFAR_DIR, batch_size=batch_size)
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        start_time = time.time()
        #batch_data, batch_labels = cifar10_input.distorted_inputs(data_dir=CIFAR_DIR,batch_size=batch_size)
        batch_data, batch_labels = train_data.next_batch(batch_size)
        #batch_data, batch_labels = sess.run([batch_data, batch_labels ])
        #image_array = np.array(batch_data)
        #image_array =image_array.astype(float)
        #batch_data = np.reshape(image_array,[20,32,32,3])        
        loss_val , acc_val,_ = sess.run(
            [loss,accuracy,train_op],feed_dict={
                x: batch_data,y: batch_labels,keep_prob:1.0})
        duration = time.time() - start_time
        if (i+1) % 50 == 0:
            examples_per_sec = batch_size / duration
            sec_per_batch = float(duration)
            format_str = ('step %d, lass=%.2f (%.1f examples/sec; %.3f sec/batch),acc: %4.5f')
            print(format_str % (i+1, loss_val, examples_per_sec, sec_per_batch,acc_val))
    
    
    num_examples = 100
    all_test_acc_val = []
    num_inter = int(math.ceil(num_examples / batch_size))
    true_count = 0   
    total_sample_count = num_inter * batch_size
    step = 0
    while step < num_inter:
        #test_batch_data, test_batch_labels= sess.run([test_batch_data, test_batch_labels]   ) 
        #test_batch_data = tf.reshape(test_batch_data,[20,1728]
        test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
        test_acc_val,count_pred_true =sess.run(
            [accuracy,top_k_op],
            feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        image_prediction = sess.run(
            y_,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        image_real = sess.run(
            y,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        true_count += np.sum(count_pred_true)
        #print(image_real)
        image_prediction = tf.nn.top_k(image_prediction,1)
        image_real = tf.nn.top_k(image_real,1)
        #print(sess.run(image_prediction.indices))
        print('[Test] Step:%d 真实分类是 %s，预测分类是 %s'% 
              (i+1 ,sess.run(image_prediction.indices),sess.run(image_real.indices)))
        step += 1
    prediction = true_count / total_sample_count
    print ('precision @ 1 = %.3f,acc: %4.5f '
           % (prediction,test_acc_val))
 

step 50, lass=2.14 (192.7 examples/sec; 0.104 sec/batch),acc: 0.25000
step 100, lass=2.22 (194.1 examples/sec; 0.103 sec/batch),acc: 0.15000
step 150, lass=2.07 (193.7 examples/sec; 0.103 sec/batch),acc: 0.15000
step 200, lass=1.85 (196.4 examples/sec; 0.102 sec/batch),acc: 0.30000
step 250, lass=2.07 (190.7 examples/sec; 0.105 sec/batch),acc: 0.25000
step 300, lass=2.14 (187.5 examples/sec; 0.107 sec/batch),acc: 0.20000
step 350, lass=1.89 (195.3 examples/sec; 0.102 sec/batch),acc: 0.45000
step 400, lass=1.63 (195.2 examples/sec; 0.102 sec/batch),acc: 0.35000
step 450, lass=2.05 (194.2 examples/sec; 0.103 sec/batch),acc: 0.40000
step 500, lass=1.91 (192.3 examples/sec; 0.104 sec/batch),acc: 0.35000
step 550, lass=1.64 (194.8 examples/sec; 0.103 sec/batch),acc: 0.50000
step 600, lass=1.71 (195.1 examples/sec; 0.103 sec/batch),acc: 0.40000
step 650, lass=1.68 (192.8 examples/sec; 0.104 sec/batch),acc: 0.35000
step 700, lass=1.36 (192.9 examples/sec; 0.104 sec/batch),acc: 0.55000
step 75

In [ ]:
   
"""    
   all_test_acc_val = []
    for j in range(test_steps):
        test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
        test_acc_val = sess.run(accuracy,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        #image_prediction = sess.run(y_,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        #image_real = sess.run(y,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        #image_prediction = tf.nn.top_k(image_prediction,1)
        #image_real = tf.nn.top_k(image_real,1)        
        all_test_acc_val.append(test_acc_val)
    test_acc = np.mean(all_test_acc_val)
    #print('[Test] Step:%d 真实分类是 %s，预测分类是 %s, acc: %4.5f'% (i+1 ,image_prediction.values,image_real.values,test_acc))
    print('[Test] Step:%d , acc: %4.5f'% (i+1 ,test_acc))
     
"""    
